In [10]:
!pip install requests pytz

import os
import requests
from datetime import datetime
import pytz

API_KEY = "bb4287f5749017257bd0e700a5e6ab1c"

os.environ["AVIATIONSTACK_API_KEY"] = API_KEY

api_key = os.getenv("AVIATIONSTACK_API_KEY")

tail_numbers = ["N350XX", "N621MM", "N143MW"]

BASE_URL = "http://api.aviationstack.com/v1/flights"

def convert_utc_to_local(utc_time):
    if utc_time:
        try:
            utc_dt = datetime.strptime(utc_time, "%Y-%m-%dT%H:%M:%S%z")
            local_dt = utc_dt.astimezone(pytz.timezone("America/New_York"))
            return local_dt.strftime("%Y-%m-%d %I:%M %p %Z")
        except Exception as e:
            return f"Time Conversion Error: {e}"
    return "Unknown"

def get_flight_info(tail_number):
    params = {
        "access_key": api_key,
        "reg_number": tail_number
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        flight_data = response.json().get("data", [])
        if flight_data:
            last_flight = flight_data[0]
            return {
                "Tail Number": tail_number,
                "Departure Airport": last_flight["departure"]["airport"],
                "Arrival Airport": last_flight["arrival"]["airport"],
                "Departure Time": convert_utc_to_local(last_flight["departure"]["estimated"]),
                "Arrival Time": convert_utc_to_local(last_flight["arrival"]["estimated"]),
                "Flight Status": last_flight["flight_status"],
                "Aircraft Type": last_flight["aircraft"]["icao_code"] if last_flight.get("aircraft") else "Unknown"
            }
        else:
            return {"Tail Number": tail_number, "Error": "No recent flights found"}
    else:
        return {"Tail Number": tail_number, "Error": f"API Error {response.status_code}: {response.text}"}

flight_results = [get_flight_info(tail) for tail in tail_numbers]

for result in flight_results:
    print("\nFlight Info")
    for key, value in result.items():
        print(f"{key}: {value}")



Flight Info
Tail Number: N350XX
Departure Airport: Gold Coast (coolangatta)
Arrival Airport: Coffs Harbour
Departure Time: 2025-02-26 07:00 AM EST
Arrival Time: 2025-02-25 10:00 AM EST
Flight Status: scheduled
Aircraft Type: Unknown

Flight Info
Tail Number: N621MM
Departure Airport: Gold Coast (coolangatta)
Arrival Airport: Coffs Harbour
Departure Time: 2025-02-26 07:00 AM EST
Arrival Time: 2025-02-25 10:00 AM EST
Flight Status: scheduled
Aircraft Type: Unknown

Flight Info
Tail Number: N143MW
Departure Airport: Gold Coast (coolangatta)
Arrival Airport: Coffs Harbour
Departure Time: 2025-02-26 07:00 AM EST
Arrival Time: 2025-02-25 10:00 AM EST
Flight Status: scheduled
Aircraft Type: Unknown
